# _**BEDU** | DATA SCIENCE (FASE 2)_
## _**MÓDULO 2** | PROCESAMIENTO DE DATOS CON PYTHON_
#### _**PROYECTO FINAL** | Colab NOTEBOOK (EQUIPO 8)_

**¡ANTES DE COMENZAR!**

Los conjuntos de datos que se utilizan a lo largo de este _Notebook_ se obtienen en tiempo real mediante solicitudes `HTTP`, es decir se utiliza una `API` **[(visita el enlace)](https://gisumd.github.io/COVID-19-API-Documentation/)** de la cual se extrae, poco a poco, un _dataset_ personalizado. 

Esta `API` ha sido elaborada sinérgicamente por Facebook y la Universidad de Maryland (Estados Unidos, Washington D. C.); está segmentada y, por tanto, la extracción de datos se segmenta también. No obstante, diversas técnicas programacionales han de ser implementadas para construir y personalizar un _dataset_ que contenga la información que más nos interese.

Sépase que los conjuntos de datos que almacena esta `API` se obtuvieron de encuestas que se aplican diariamente, sin embargo, la actualización del _datset_ original no es realizada con la misma frecuencia.

AHORA SÍ... **¡A COMENZAR!**

In [ ]:
# Se importan los paquetes y las librerías que se utilizarán dentro de este 'Notebook'.
import pandas as pd
import numpy as np
import requests

#### _1. Acceso a la `API` de la Universidad de Maryland y Facebook_

###### _1.1. Accediendo al conjunto de datos original y familiarizándonos con él_

In [ ]:
endpoint = "https://covidmap.umd.edu/api/country"
r = requests.get(endpoint)
json = r.json()
data = json["data"]
country = pd.DataFrame.from_dict(data) # A partir de una consulta 'HTTP' y con ayuda de la librería 'pandas', se genera un 'DataFrame' con los nombres de
                                       # los países que han participado en este estudio.

In [ ]:
country.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en la celda anterior.

,country
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Argentina


Es debido mencionar que específicamente nos interesa conocer si México es uno de los países que participó en la resolución de encuestas de este estudio. Por lo tanto, habrá que ejecutar la siguiente línea de código para determinarlo:

In [ ]:
"Mexico" in country.values # Si el resultado de esta sentencia es 'True', quiere decir que vamos bien y podemos continuar.

True

Entonces, continuemos familiarizándonos con nuestro conjunto de datos y el acceso a la `API`.

In [ ]:
endpoint = "https://covidmap.umd.edu/api/region"
r = requests.get(endpoint)
json = r.json()
data = json["data"]
country_region = pd.DataFrame.from_dict(data) # Nuevamente, a partir de una consulta 'HTTP' y con ayuda de la librería 'pandas', se genera un 'DataFrame' con
                                              # los nombres de los países participantes, pero esta vez se anexan las regiones, por país, que aportaron datos 
                                              # al estudio.

In [ ]:
country_region.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en la celda anterior.

,country,region
0,Afghanistan,Badakhshan
1,Afghanistan,Balkh
2,Afghanistan,Kabul
3,Albania,Tiranë
4,Algeria,Alger


Una vez más, nótese que para nuestro análisis solamente atañe la información que proporciona México, por ello se aplica el siguiente filtro:

In [ ]:
mexico_data = country_region[country_region["country"] == "Mexico"].reset_index()
mexico_data = mexico_data.loc[:, "country":]
mexico_data # Se obtiene un 'DataFrame' que contiene únicamente las regiones de México que participaron dentro del estudio (es decir, aquellas
            # cuyos habitantes contestaron la encuesta).

,country,region
0,Mexico,Aguascalientes
1,Mexico,Baja California
2,Mexico,Baja California Sur
3,Mexico,Campeche
4,Mexico,Chiapas
5,Mexico,Chihuahua
6,Mexico,Coahuila
7,Mexico,Colima
8,Mexico,Distrito Federal
9,Mexico,Durango


In [ ]:
mexico_data.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en la celda anterior.

,country,region
0,Mexico,Aguascalientes
1,Mexico,Baja California
2,Mexico,Baja California Sur
3,Mexico,Campeche
4,Mexico,Chiapas


###### _1.2. Generando nuestro propio `dataset` a partir del `dataset` original_

In [ ]:
data_range = list(mexico_data["region"]) # Esta estructura de datos contiene los nombres de las regiones de México que participaron en el estudio.
concat_data = pd.Series(np.nan)

for i in data_range:
  try:
    # time.sleep(5) # La teoría aconseja que se dejen pasar cinco segundos entre solicitud y solicitud para que no se origine ningún problema de comunicación
    # con el servidor, sin embargo, en esta ocasión se omitió dicha recomendación, pues el tiempo de ejecución de este segmento de código se volvería bastante
    # prolongado si se hiciera caso.
    
    endpoint = "https://covidmap.umd.edu/api/datesavail?country=Mexico&region=" + i
    r = requests.get(endpoint)

    if r.status_code == 200:
      json = r.json()
      data = json["data"] # Nótese que en este caso no es necesario normalizar el archivo 'JSON'.
      country_region_surveydate = pd.DataFrame.from_dict(data)

      concat_data = pd.concat([concat_data, country_region_surveydate], axis = 0)
      half_full_mexico_data = concat_data.reset_index(drop = True) # Se obtiene un 'DataFrame' concatenado que contiene las regiones en donde se aplicó la
                                                                   # encuesta y sus respectivas fechas de aplicación.
  except:
    print("¡Nótese que hubo un error dentro de la ejecución de este código!")
    continue

half_full_mexico_data = half_full_mexico_data.loc[1:, "country":]

In [ ]:
min_date = half_full_mexico_data["survey_date"].min() # Con esta línea de código se identifica la fecha en que México comenzó a proporcionar datos.
max_date = half_full_mexico_data["survey_date"].max() # Con esta otra línea se conoce la úlitma fecha en la que una encuesta fue respondida.

In [ ]:
indicators = ["covid", "mask", "finance", "food_security", "vaccine_acpt", "covid_vaccine", "twodoses", "cmty_covid", "trust_fam", "trust_healthcare",
              "trust_who", "trust_govt", "trust_politicians", "barrier_reason_side_effects", "barrier_reason_dontbelieve", "barrier_reason_dontlike",
              "barrier_reason_government", "barrier_reason_dontneed_alreadyhad", "barrier_reason_dontneed_notbeneficial", "vaccine_barrier_time",
              "vaccine_barrier_type"] # Esta lista contiene los indicadores que consideramos más representativos y con los cuales
                                      # construiremos nuestro 'dataset'. Recuérdese que nuestro objetivo principal es: determinar la
                                      # postura ideológica predominante entre la población mexicana hacia el fenómeno de vacunación COVID-19.
indicators_data = []

for i in indicators:
  try:
    endpoint = "https://covidmap.umd.edu/api/resources?indicator=" + i + "&type=daily&country=Mexico&daterange=" + min_date + "-" + max_date
    r = requests.get(endpoint)
    
    if r.status_code == 200:
      json = r.json()
      data = json["data"] # Nótese que nuevamente no es necesario normalizar el archivo 'JSON'.
      indicator = pd.DataFrame.from_dict(data)
      indicator = indicator[[indicator.columns[0], "sample_size", "survey_date"]]
      indicators_data.append(indicator) # Se rescatan las columnas de interés y se almacenan 'DataFrames' en una lista por si algún análisis futuro requiriera
                                        # los datos de cada indicador por separado.
      half_full_mexico_data = half_full_mexico_data.merge(indicator, how = "outer", on = "survey_date") # Se aplica el método 'merge' para unir toda la
                                                                                                        # información en un solo 'DataFrame'.
  except:
    print("¡Nótese que hubo un error dentro de la ejecución de este código!")
    continue

###### _1.3. Explorando, reindexando, renombrando y limpiando nuestro `dataset`_

In [ ]:
half_full_mexico_data = half_full_mexico_data.sort_values(["region", "survey_date"],
                                                          ascending = True,
                                                          ignore_index = True).loc[:, "region":] # Se ordena el 'DataFrame' resultante, por región
                                                                                                 # y fecha de aplicación.

In [ ]:
cols = pd.MultiIndex.from_tuples([("survey_information", "region_where_applied"), ("survey_information", "survey_date"),
                                  ("covid_19", "indicator_percentage"), ("covid_19", "sample_size"),
                                  ("wear_a_mask", "indicator_percentage"), ("wear_a_mask", "sample_size"),
                                  ("financial_concern", "indicator_percentage"), ("financial_concern", "sample_size"),
                                  ("food_lackness_concern", "indicator_percentage"), ("food_lackness_concern", "sample_size"),
                                  ("accept_vaccine", "indicator_percentage"), ("accept_vaccine", "sample_size"),                                  
                                  ("partially_vaccinated", "indicator_percentage"), ("partially_vaccinated", "sample_size"),
                                  ("fully_vaccinated", "indicator_percentage"), ("fully_vaccinated", "sample_size"),
                                  ("know_people_sick", "indicator_percentage"), ("know_people_sick", "sample_size"),
                                  ("trust_family_recommendation", "indicator_percentage"), ("trust_family_recommendation", "sample_size"),
                                  ("trust_healthcare_recommendation", "indicator_percentage"), ("trust_healthcare_recommendation", "sample_size"),
                                  ("trust_WHO_recommendation", "indicator_percentage"), ("trust_WHO_recommendation", "sample_size"),
                                  ("trust_government_recommendation", "indicator_percentage"), ("trust_government_recommendation", "sample_size"),
                                  ("trust_politicians_recommendation", "indicator_percentage"), ("trust_politicians_recommendation", "sample_size"),
                                  ("barrier_side_effects", "indicator_percentage"), ("barrier_side_effects", "sample_size"),
                                  ("barrier_dont_believe", "indicator_percentage"), ("barrier_dont_believe", "sample_size"),
                                  ("barrier_dont_like_vaccines", "indicator_percentage"), ("barrier_dont_like_vaccines", "sample_size"),
                                  ("barrier_dont_trust_government", "indicator_percentage"), ("barrier_dont_trust_government", "sample_size"),
                                  ("barrier_dont_need_already_had", "indicator_percentage"), ("barrier_dont_need_already_had", "sample_size"),
                                  ("barrier_dont_need_not_beneficial", "indicator_percentage"), ("barrier_dont_need_not_beneficial", "sample_size"),
                                  ("barrier_lack_of_time", "indicator_percentage"), ("barrier_lack_of_time", "sample_size"),
                                  ("barrier_vaccine_type", "indicator_percentage"), ("barrier_vaccine_type", "sample_size")])

half_full_mexico_data.columns = cols # Se renombran las columnas del 'DataFrame' que hemos procesado hasta el momento, aplicando el método de
                                     # multi-indexación por columnas.

El _dataset_ original contiene más indicadores, pero solamente extrajimos aquellos que, según nuestras preguntas u objetivos [**(consúltese el trabajo de investigación que desarrollamos)**](https://docs.google.com/document/d/1wwfUJ4ZAmH6cBWL_miKNlR_3_5xI5iKGUcqG-Ijl-J4/edit), se creyó eran los más indicados para nosotros.

In [ ]:
half_full_mexico_data.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en la celda de código anterior.

survey_information              ... barrier_vaccine_type            
  region_where_applied survey_date  ... indicator_percentage sample_size
0       Aguascalientes    20200501  ...                  NaN         NaN
1       Aguascalientes    20200502  ...                  NaN         NaN
2       Aguascalientes    20200503  ...                  NaN         NaN
3       Aguascalientes    20200504  ...                  NaN         NaN
4       Aguascalientes    20200505  ...                  NaN         NaN

[5 rows x 44 columns]

In [ ]:
pd.isna(half_full_mexico_data).sum() # Con esta línea de código se puede observar la cantidad de 'NaNs' que posee nuestro 'DataFrame'.

survey_information                region_where_applied        0
                                  survey_date                 0
covid_19                          indicator_percentage        0
                                  sample_size                 0
wear_a_mask                       indicator_percentage       23
                                  sample_size                23
financial_concern                 indicator_percentage       74
                                  sample_size                74
food_lackness_concern             indicator_percentage     5664
                                  sample_size              5664
accept_vaccine                    indicator_percentage     7607
                                  sample_size              7607
partially_vaccinated              indicator_percentage     6843
                                  sample_size              6843
fully_vaccinated                  indicator_percentage     7112
                                  sample

**Nótese** que hay muchas filas con bastantes valores nulos. Esto se debe a que las encuestas aplicadas han tenido actualizaciones a lo largo del tiempo, lo que significa que no todos los indicadores existieron desde el principio y tampoco todos han prevalecido hasta hoy.

Por otro lado, en algunos casos no todos los encuestados respondieron todas las preguntas, fenómeno que se ve reflejado en las columnas subindexadas con el nombre `sample_size`; claramente se observa cómo los tamaños muestrales varían entre sí aun cuando la fecha y región de aplicación son las mismas.

In [ ]:
half_full_mexico_data.info() # Con esta línea de código se obtiene información acerca de nuestro 'DataFrame'. Obsérvese que el método 'info()'
                             # es aun más descriptivo que simplemente aplicar 'pd.isna().sum()', pues en él se incluye la tipología de todas las variables.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11997 entries, 0 to 11996
Data columns (total 44 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   (survey_information, region_where_applied)                11997 non-null  object 
 1   (survey_information, survey_date)                         11997 non-null  object 
 2   (covid_19, indicator_percentage)                          11997 non-null  float64
 3   (covid_19, sample_size)                                   11997 non-null  float64
 4   (wear_a_mask, indicator_percentage)                       11974 non-null  float64
 5   (wear_a_mask, sample_size)                                11974 non-null  float64
 6   (financial_concern, indicator_percentage)                 11923 non-null  float64
 7   (financial_concern, sample_size)                          11923 non-null  float64
 8   (food_lackness_c

Nuestro dataset así como está contiene 11997 entradas en total, las cuales son suficientes para llevar a cabo nuestro proyecto. Estas contienen información pertinente a la percepción que tiene el público mexicano en cuanto a las vacunas en distintos aspectos, así que fácilmente pueden contestar las preguntas que hemos planteado para nuestro proyecto. Del total observamos que varias contienen valores nulos en ciertas columnas, posiblemente porque no todas las preguntas se realizaron en todas las encuestas.

Cabe recalcar que cada entrada equivale a una encuesta realizada en un día en una región de México, y estás encuestas tienen diferentes tamaños de muestra. Cada columna representa una pregunta de la encuesta, la mayoría siendo preguntas de si o no. Por ejemplo, en la columna “wear_a_mask” se les pregunta a los encuestados si usan un cubrebocas la mayoría del tiempo cuando salen. Los nombres son concisos y se entienden, y optaremos por dejarlos en inglés.

#### _2. Procesamiento de datos_

Antes de continuar, parece ser que contar con un _dataset_ completamente limpio y libre de `NaNs` **no es simplemente un requisito, es más bien una actividad indispensable para continuar pre-procesando los datos.**

Ahora, pensar en eliminar los valores nulos por medio de un método convencional, por ejemplo utilizando `full_mexico_data = half_full_mexico_data.dropna()`, definitivamente no es una buena idea, pues el 'DataFrame' se vaciaría por completo. Entonces, buscaremos agrupar los datos en subconjuntos, considerando las fechas en las que los indicadores fueron publicados por primera vez en las encuestas. **Recuérdese que** las encuestas se han ido actualizando con el tiempo y no todos los inidicadores poseen la misma antiguedad, sin emabrgo sí podemos encontrar coincidencias y con base en ellas agrupar en nuevos y más pequeños 'DataFrames'.

Por otro lado, del resumen anterior `(half_full_mexico_data.info())` se puede observar que la columna que contiene las fechas de aplicación **no posee un formato adecuado**, por lo cual procederemos a modificarla. Además, las columnas subindexadas con el nombre `sample_size` tendrían un mejor formato si las cambiáramos de `float64` a `int64`.

Entonces, sin más rodeo, se procede de la siguiente manera:

In [ ]:
from datetime import datetime

subset_data_1 = half_full_mexico_data[["survey_information", "covid_19", "know_people_sick",
                                              "wear_a_mask", "financial_concern", "food_lackness_concern"]] # Se separa el primer subconjunto de datos.

subset_data_2 = half_full_mexico_data[["survey_information", "accept_vaccine", "trust_family_recommendation",
                                       "trust_healthcare_recommendation", "trust_WHO_recommendation",
                                       "trust_government_recommendation", "trust_politicians_recommendation"]] # Se separa el segundo subconjunto de datos.

subset_data_3 = half_full_mexico_data[["survey_information", "partially_vaccinated", "fully_vaccinated", "barrier_side_effects",
                                              "barrier_dont_need_already_had", "barrier_dont_need_not_beneficial",
                                              "barrier_dont_believe", "barrier_dont_like_vaccines",
                                              "barrier_dont_trust_government"]] # Se separa el tercer subconjunto de datos.

subset_data_4 = half_full_mexico_data[["survey_information", "barrier_vaccine_type", "barrier_lack_of_time"]] # Se separa el cuarto subconjunto de datos.

dty = {"subset_data_1": subset_data_1, "subset_data_2": subset_data_2, "subset_data_3": subset_data_3, "subset_data_4": subset_data_4,}

def date_year_month(survey_date):
  return str(survey_date.year) + "-" + str(survey_date.month)

def subset_settings(num):
  dty["subset_data_" + str(num[0])] = dty["subset_data_" + str(num[0])].dropna().reset_index(drop = True) # En esta línea de código se efectúa una limpieza
  # de los subconjuntos que previamente creamos (todas las filas que poseen al menos un valor 'NaN' quedan fuera de los 'DataFrames').
  dty["subset_data_" + str(num[0])].iloc[:, range(3, num[1], 2)] = dty["subset_data_" + str(num[0])].iloc[:, range(3, num[1], 2)].astype("int64")
  dty["subset_data_" + str(num[0])][("survey_information",
                                     "survey_date")] = pd.to_datetime(dty["subset_data_" + str(num[0])][("survey_information", "survey_date")]) # En las dos
                                     # líneas de código superiores se modifican los tipos de dato de aquellas columnas que así lo requieren.
  dty["subset_data_" + str(num[0])].insert(loc = 2, column = ("", "survey_date_short"),
                                           value = dty["subset_data_" + str(num[0])][("survey_information",
                                                                                      "survey_date")].apply(date_year_month), allow_duplicates = False) # Con las
                                                                                      # dos líneas de código superiores se anexa una columna a los subconjuntos
                                                                                      # de datos, la cual nos permitirá realizar agrupaciones y aplicar
                                                                                      # agregaciones más adelante.
empty_list = list(map(subset_settings, [(1, 12), (2, 14), (3, 18), (4, 6)]))

subset_data_1 = dty["subset_data_1"]; subset_data_2 = dty["subset_data_2"]; subset_data_3 = dty["subset_data_3"]; subset_data_4 = dty["subset_data_4"]

Ahora sí ya no hay `NaNs`, lo cual nos permite operar de forma libre con la matemática en los conjuntos de datos.

De modo entonces que, a partir de este momento, resulta atractivo empezar a generar agrupaciones para que, mediante el uso de agregaciones, obtengamos nuevos datos que más adelante nos permitirán desarrollar un análisis estadístico; la idea es que los nuevos datos nos sirvan como punto de partida para generar análisis estadísticos robustos y herramientas de visualización sencillas que nos brinden **un mejor entendimiento del problema.**

In [ ]:
subset_data_1.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en las celdas anteriores.

survey_information              ... food_lackness_concern            
  region_where_applied survey_date  ...  indicator_percentage sample_size
0       Aguascalientes  2020-11-23  ...              0.418631        6983
1       Aguascalientes  2020-11-24  ...              0.417379        7845
2       Aguascalientes  2020-11-25  ...              0.415340        7826
3       Aguascalientes  2020-11-26  ...              0.425219        7666
4       Aguascalientes  2020-11-27  ...              0.400846        7694

[5 rows x 13 columns]

In [ ]:
subset_data_2.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en las celdas anteriores.

survey_information              ... trust_politicians_recommendation            
  region_where_applied survey_date  ...             indicator_percentage sample_size
0       Aguascalientes  2020-12-21  ...                         0.343213        1506
1       Aguascalientes  2020-12-22  ...                         0.356461        4012
2       Aguascalientes  2020-12-23  ...                         0.361988        4070
3       Aguascalientes  2020-12-24  ...                         0.393401        3454
4       Aguascalientes  2020-12-25  ...                         0.402814        3439

[5 rows x 15 columns]

In [ ]:
subset_data_3.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en las celdas anteriores.

survey_information              ... barrier_dont_trust_government            
  region_where_applied survey_date  ...          indicator_percentage sample_size
0       Aguascalientes  2021-02-06  ...                           0.0        2551
1       Aguascalientes  2021-02-07  ...                           0.0        2437
2       Aguascalientes  2021-02-08  ...                           0.0        2560
3       Aguascalientes  2021-02-09  ...                           0.0        2627
4       Aguascalientes  2021-02-10  ...                           0.0        2656

[5 rows x 19 columns]

In [ ]:
subset_data_4.head() # Ejecuta esta celda únicamente si deseas observar el resultado de lo que se escribió en las celdas anteriores.

survey_information              ... barrier_lack_of_time            
  region_where_applied survey_date  ... indicator_percentage sample_size
0       Aguascalientes  2021-07-20  ...             0.045544        5935
1       Aguascalientes  2021-07-26  ...             0.046705        6328
2       Aguascalientes  2021-07-29  ...             0.043467        6261
3       Aguascalientes  2021-08-03  ...             0.041087        6948
4       Aguascalientes  2021-08-04  ...             0.049977        6975

[5 rows x 7 columns]

#### _3. Agrupación, agregaciones y presentación de los primeros resultados_

###### _3.1. Agrupación y agregaciones_

In [ ]:
# Agrupación 1 (aplica para todos los subconjuntos):
# Con esta agrupación y la agregación 'count()' se puede determinar el número de días, por mes y región, en los que los indicadores de cada subconjunto
# tuvieron respuestas.
sub1_agg_1 = subset_data_1.groupby([("survey_information", "region_where_applied"),
                                    ("", "survey_date_short")])[[("survey_information", "region_where_applied")]].agg("count")
sub1_agg_1.columns = ["days_of_survey_a_month_per_region"]
sub1_agg_1.index.names = ["region", "short_date"]

sub2_agg_1 = subset_data_2.groupby([("survey_information", "region_where_applied"),
                                    ("", "survey_date_short")])[[("survey_information", "region_where_applied")]].agg("count")
sub2_agg_1.columns = ["days_of_survey_a_month_per_region"]
sub2_agg_1.index.names = ["region", "short_date"]

sub3_agg_1 = subset_data_3.groupby([("survey_information", "region_where_applied"),
                                    ("", "survey_date_short")])[[("survey_information", "region_where_applied")]].agg("count")
sub3_agg_1.columns = ["days_of_a_month_per_region"]
sub3_agg_1.index.names = ["region", "short_date"]

sub4_agg_1 = subset_data_4.groupby([("survey_information", "region_where_applied"),
                                    ("", "survey_date_short")])[[("survey_information", "region_where_applied")]].agg("count")
sub4_agg_1.columns = ["days_of_survey_a_month_per_region"]
sub4_agg_1.index.names = ["region", "short_date"]

In [ ]:
sub1_agg_1.head() # Ejecuta esta línea de código únicamente si deseas observar uno de los resultados de la 'agregación 1'.

days_of_survey_a_month_per_region
region         short_date                                   
Aguascalientes 2020-11                                     8
               2020-12                                    31
               2021-1                                     31
               2021-2                                     27
               2021-3                                     18

In [ ]:
# Agrupación 2 (aplica para todos los subconjuntos):
# Con esta agrupación y la agregación 'size()' se puede determinar el número de días, por región, en los que los indicadores de cada subconjunto
# tuvieron respuestas.
sub1_agg_2 = subset_data_1.groupby([("survey_information", "region_where_applied")])[[("survey_information", "region_where_applied")]].size()
sub1_agg_2.columns = ["days_of_survey_a_month_per_region"]
sub1_agg_2.index.names = ["region"]

sub2_agg_2 = subset_data_2.groupby([("survey_information", "region_where_applied")])[[("survey_information", "region_where_applied")]].size()
sub2_agg_2.columns = ["days_of_survey_a_month_per_region"]
sub2_agg_2.index.names = ["region"]

sub3_agg_2 = subset_data_3.groupby([("survey_information", "region_where_applied")])[[("survey_information", "region_where_applied")]].size()
sub3_agg_2.columns = ["days_of_a_month_per_region"]
sub3_agg_2.index.names = ["region"]

sub4_agg_2 = subset_data_4.groupby([("survey_information", "region_where_applied")])[[("survey_information", "region_where_applied")]].size()
sub4_agg_2.columns = ["days_of_survey_a_month_per_region"]
sub4_agg_2.index.names = ["region"]

In [ ]:
sub1_agg_2.head() # Ejecuta esta línea de código únicamente si deseas observar uno de los resultados de la 'agregación 2'.

region
Aguascalientes         133
Baja California        261
Baja California Sur     68
Campeche                 3
Chiapas                209
dtype: int64

In [ ]:
# Agrupación 3 (aplica para todos los subconjuntos):
# Con esta agrupación y la agregación 'sum()' se puede determinar el número de personas encuestadas, por día e indicador.
sub1_agg_3 = subset_data_1.groupby([("survey_information", "survey_date")]).agg("sum")
sub1_agg_3 = sub1_agg_3.iloc[:, range(1, 10, 2)]
sub1_agg_3.columns = ["answers_a_day_covid19", "answers_a_day_know_people_sick", "answers_a_day_wear_a_mask",
                      "answers_a_day_financial_concern", "answers_a_day_food_lackness_concern"]
sub1_agg_3.index.names = ["long_date"]

sub2_agg_3 = subset_data_2.groupby([("survey_information", "survey_date")]).agg("sum")
sub2_agg_3 = sub2_agg_3.iloc[:, range(1, 12, 2)]
sub2_agg_3.columns = ["answers_a_day_accept_vaccine", "answers_a_day_trust_family", "answers_a_day_trust_healthcare",
                      "answers_a_day_trust_WHO", "answers_a_day_trust_government", "answers_a_day_trust_politicians"]
sub2_agg_3.index.names = ["long_date"]

sub3_agg_3 = subset_data_3.groupby([("survey_information", "survey_date")]).agg("sum")
sub3_agg_3 = sub3_agg_3.iloc[:, range(1, 16, 2)]
sub3_agg_3.columns = ["answers_a_day_partially_vaccinated", "answers_a_day_fully_vaccinated", "answers_a_day_side_effects",
                      "answers_a_day_already_had", "answers_a_day_not_benefical", "answers_a_day_dont_believe",
                       "answers_a_day_dont_like_vaccines", "answers_a_day_dont_trust_government"]
sub3_agg_3.index.names = ["long_date"]

sub4_agg_3 = subset_data_4.groupby([("survey_information", "survey_date")]).agg("sum")
sub4_agg_3 = sub4_agg_3.iloc[:, range(1, 4, 2)]
sub4_agg_3.columns = ["answers_a_day_vaccine_type", "answers_a_day_lack_of_time"]
sub4_agg_3.index.names = ["long_date"]

In [ ]:
sub1_agg_3.head() # Ejecuta esta línea de código únicamente si deseas observar uno de los resultados de la 'agregación 3'.

,answers_a_day_covid19,answers_a_day_know_people_sick,answers_a_day_wear_a_mask,answers_a_day_financial_concern,answers_a_day_food_lackness_concern
long_date,,,,,
2020-11-23,228231,214812,201285,188757,188541
2020-11-24,265720,251468,233912,219884,219660
2020-11-25,257580,241542,225909,211464,211302
2020-11-26,251505,236898,221535,207252,206982
2020-11-27,254610,239571,222912,208170,207738


###### _3.2. Presentación de los primeros resultados_

Hasta el momento ha quedado demostrado que se pueden hacer agrupaciones que regresan información relevante, las cuales eventualmente nos podrían proporcionar más información. Sin embargo, las agrupaciones hechas hasta ahora se han proyectado a nivel general, es decir considerando todos los indicadores, de cada subconjunto, a la vez.

Para concluir con el desarrollo de todo esto, haremos una última agrupación en donde apliquemos, sobre un identificador en específico, más de una agregación. Con ello se pretende responder preguntas algo más interesantes. Por ejemplo, piénsese en lo siguiente...

Desde el comienzo de los tiempos en los que se aplicó la encuesta en México,

*   **¿cuál es el porcentaje de aceptación más alto que ha tenido la vacuna? ¿Cuándo y en qué región se presentó este fenómeno?**
*   **¿cuál es el porcentaje de aceptación más bajo que ha tenido la vacuna? ¿Cuándo y en qué región se presentó este fenómeno?**
*   **¿cuál ha sido el promedio de aceptación más alto que ha tenido la vacuna? ¿Cuándo y dónde ocurrió este fenómeno?**
*   **¿cuál ha sido el promedio de aceptación más bajo que ha tenido la vacuna? ¿Cuándo y dónde ocurrió este fenómeno?**

In [ ]:
# Agrupación 4 (aplica para el indicador 'aceptación hacia la vacuna COVID19', ubicado en el segundo subconjunto de datos):
# Se agrupará por regiones y por fechas cortas (año y mes únicamente) para aplicar distintas agregaciones que nos permitan conocer valores estadísticos
# y responder preguntas de interés.
sub1_agg_4 = subset_data_2.groupby([("survey_information", "region_where_applied"),
                                    ("", "survey_date_short")])[[("accept_vaccine",
                                                                  "indicator_percentage")]].agg(["median", "mean", "var", "std", "max", "min"])
sub1_agg_4.columns = ["median", "mean", "variance", "standard_deviation", "maximum", "minimum"]
sub1_agg_4.index.names = ["region", "date_short"]

In [ ]:
sub1_agg_4.head() # Ejecuta esta línea de código únicamente si deseas observar el resultado de la 'agregación 4'.

median      mean  ...   maximum   minimum
region          date_short                      ...                    
Aguascalientes  2020-12     0.845975  0.842025  ...  0.854653  0.820356
                2021-1      0.865574  0.858766  ...  0.881077  0.824280
                2021-2      0.885686  0.884344  ...  0.894202  0.867286
Baja California 2020-12     0.845975  0.842025  ...  0.854653  0.820356
                2021-1      0.865574  0.858766  ...  0.881077  0.824280

[5 rows x 6 columns]

In [ ]:
max_acceptance = sub1_agg_4[sub1_agg_4["maximum"].max() == sub1_agg_4["maximum"]] # Esta línea de código responde el primer grupo de preguntas.
min_acceptance = sub1_agg_4[sub1_agg_4["minimum"].min() == sub1_agg_4["minimum"]] # Esta línea de código responde el segundo grupo de preguntas.
max_avg_acceptance = sub1_agg_4[sub1_agg_4["mean"].max() == sub1_agg_4["mean"]] # Esta línea de código responde el tercer grupo de preguntas.
min_avg_acceptance = sub1_agg_4[sub1_agg_4["mean"].min() == sub1_agg_4["mean"]] # Esta línea de código responde el cuarto grupo de preguntas.

In [ ]:
from pprint import pprint
# Ejecuta el siguiente segmento de código únicamente si deseas conocer las respuestas a las preguntas planteadas previamente.
pprint(max_acceptance) # Máxima aceptación.
pprint(min_acceptance) # Mínima aceptación.
pprint(max_avg_acceptance) # Promedio de aceptación máximo.
pprint(min_avg_acceptance) # Promedio de aceptación mínimo.

                                  median      mean  ...   maximum   minimum
region              date_short                      ...                    
Aguascalientes      2021-2      0.885686  0.884344  ...  0.894202  0.867286
Baja California     2021-2      0.885686  0.884344  ...  0.894202  0.867286
Baja California Sur 2021-2      0.885180  0.884903  ...  0.894202  0.879802
Chiapas             2021-2      0.885686  0.884344  ...  0.894202  0.867286
Chihuahua           2021-2      0.885686  0.884344  ...  0.894202  0.867286
Coahuila            2021-2      0.885686  0.884344  ...  0.894202  0.867286
Colima              2021-2      0.890229  0.890229  ...  0.894202  0.886255
Distrito Federal    2021-2      0.885686  0.884344  ...  0.894202  0.867286
Durango             2021-2      0.885686  0.884344  ...  0.894202  0.867286
Guanajuato          2021-2      0.885686  0.884344  ...  0.894202  0.867286
Guerrero            2021-2      0.885686  0.884344  ...  0.894202  0.867286
Hidalgo     

**Planes para análisis posterior**

Después de una extenuante exploración y procesamiento iniciales, es correcto 
afirmar que la información obtenida es de utilidad para un análisis más 
profundo orientado a encontrar patrones que provean un rumbo hacia un plan de acción para resolver el problema de inicio. 

Al haber segmentado los indicadores por afinidad, a partir de este punto es posible establecer una correlación de cualquiera de ellos con el porcentaje de aceptación a lo largo del tiempo. Recordando el carácter multi factor de la problemática, un objetivo para la siguiente fase es identificar aquellos indicadores más determinantes en la posición respecto a la vacuna. 

Una correlación de gran interés es la que existe entre el grado de aceptación y el porcentaje de inmunización completa de la población. O bien, por otro lado, encontrar una relación significativa entre indicadores. Por ejemplo, se tiene en mente conocer el vínculo entre el grado de preocupación financiera y haber recibido ambas dosis de la vacuna. 

Es importante conocer la opinión pública que se desarrolla en plena pandemia, con la exploración de datos ya recabada nos permitirá evaluar el nivel de influencia social con respecto a la toma de la vacuna, dependiendo de aspectos categóricos midiendo el impacto social de las distintas regiones, las marginadas y las no marginada.

En determinado momento resultará necesario obtener gráficas para la visualización del comportamiento de las interacciones que se establezcan. Se tiene pensado recurrir a histogramas para observar la distribución de los datos por regiones. Incluso se puede contemplar una representación cromática de las tendencias principales sobre el mapa de la República. 

Siendo muy aventurados, es tentativo desarrollar un modelo de predicción para determinar el comportamiento de una población con características diferentes y encontrar los puntos de ajuste a factores predominantes para incidir indirectamente en la aceptación de vacunas.

Ejecutando estas acciones e incorporando sugerencias de los expertos, consideramos que se puede obtener información relevante para ser aplicada con un nuevo enfoque. Sin bien es cierto que México no destaca por su manejo de la pandemia, el éxito de sus políticas de vacunación se refleja en una alta disposición y confianza a recibir la vacuna. 

Independientemente de las particularidades de su contexto, el posterior análisis de la información procesada tiene potencial de servir como referencia para reforzar la misión a nivel global. 


**¡ANTES DE FINALIZAR!** Nótese que el tipo de procesamiento aplicado en la última agrupación se podría reproducir sobre otros indicadores para obtener más información y enriquecer nuestro estudio, de igual forma se podrían relacionar resultados entre indicadores para inferir nuevo conocimiento. No obstante, el alcance de este módulo nos permite llegar únicamente hasta este punto.

Entonces, sin más que añadir por el momento, concluimos nuestro **procesamiento de datos** y nos despedimos cálidamente.